In [1]:
import time
import os

import pandas as pd
import pickle

import requests
from pymongo import MongoClient

import spotipy

### Connecting to MongoDB

In [2]:
client = MongoClient()

In [3]:
db = client.spotify

In [4]:
db.list_collection_names()

['artists', 'albums', 'track_features', 'tracks']

In [5]:
cursor = db.tracks.find({}, 
    {'_id':0, 'album_uri':1, 'artist_id':1, 'artist_name':1, 'artist_uri':1, 'duration_ms':1, 'explicit':1, 'id':1, 'name':1, 'track_number':1, 'uri':1, 'song_spotify_page':1})
df = pd.DataFrame(list(cursor))
df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,name,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


## Removing non-queried artists

In [6]:
returned_artists = df.artist_name.unique().tolist()

In [7]:
pickle_in = open('Data/indie_pop_artists','rb')
queried_artists = pickle.load(pickle_in)
pickle_in.close()

In [8]:
print(len(queried_artists))
print(len(returned_artists))
print(len(set(queried_artists).intersection(set(returned_artists))))

716
714
539


In [9]:
artists_df = pd.DataFrame(list(set(queried_artists).intersection(set(returned_artists))))
artists_df.columns = ['artist_name']
artists_df.head()

,artist_name
0,Melanie Martinez
1,Il Genio
2,Sarah P.
3,Zella Day
4,The Submarines


In [10]:
df = df.merge(artists_df,on='artist_name')
df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,name,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


In [11]:
df.shape

(35538, 11)

In [12]:
df.artist_name.nunique()

539

## Get Track Features

In [31]:
def get_track_features(album_uri_s_list, df, access_token):
    
    sp = spotipy.Spotify(auth=access_token)
    
    for album_uri in album_uri_s_list[2:]:
        track_info = df[df.album_uri == album_uri][['artist_uri','uri']]
        
        # Arbitrarily takes the first artist_uri. They are all identical.
        artist_uri = track_info.artist_uri.values[0]
        
        track_uri_s_list = track_info.uri.values.tolist()
        # there's a hard limit of 50 queires so we limit the list size to that
        if len(track_uri_s_list) > 50:
            track_uri_s_list = track_uri_s_list[:50]
        else: 
            pass
        track_features_list = sp.audio_features(track_uri_s_list)
        
        save_features_to_mongoDB(track_features_list, artist_uri, album_uri)

In [25]:
def save_features_to_mongoDB(track_features_list, artist_uri, album_uri):
        
    for track_features in track_features_list:
        track_features['album_uri'] = album_uri
        track_features['artist_uri'] = artist_uri
        db.track_features.insert_one(track_features)

In [19]:
access_token = 'BQDHfz2qFWVynG-7Zkl4aTpb_84H3B-mOwQT1oo5sKYuuFttyB4s5liYNla92lv4G17zFrtEmu8tJxAeXPRQe0rDgjOvXbUonfrVK7wqYfbkfjqTurmeOn2Ho6oCyz44CRGKqCJtcEQhCA'

In [16]:
album_uri_s_list = df.album_uri.unique().tolist()

In [17]:
len(album_uri_s_list)

2645

In [32]:
get_track_features(album_uri_s_list, df, access_token)

In [29]:

lst = '6zu5IpVoUjkk2IxkMun7nt,3Hy3ZtmCyKxoeQUsY0MO2X,1b3X9NzCvsgc0xL2SZIZCY,32e4oNLzZCyUxBXFKUeRDl,5EMjzHxtktHefF2pI4odyg,0lYMDHsZEpqi6fLz1rHC9t,399bk7a2IULJuHyojWS4TE,1H9YGNxPbJd7cDlgW5TWoq,7FIdVmdGOeLqhPhg896Uyk,1ZDf0KAV0QnL91dMczIpoF,2WMmZwFwljXEEb235SzCfe,2HNQcYy9ap9mwm8rUaulAr,1khZhKjjqTko1IMIGXcRzV,3dkftGFCHe6lDKYBm0okDc,2ezu2y24ItRN279Iq9eeKa,2v4djdojQ5EHr82D5zEJkL,17PlNCENNIA3RvvPmmSQCJ,3W6xf9rTqvji3uXXfZ4PjG,7vDwczuymUS24PvdC5jvcA,736CpyFUJARSDyDRGXzJgd,5ZAi3u1GHPcjdVX3L9aY2z,6sm8CMElVYxd776gt3bfty,1OzZCPPwWKcjGr10KUoUA7,6kUG6EZ1d5h61js6lQKxOG,7KUBbTi3f1znp9Z51egHWx,1vhlMdyL3USJTT1IMXKa1S,2SvdE27onf1ddcJfn2KiSq,0ZN9kStb4G3g762qBEXK7H,41F3sxzQkKQOgbCDUYKY73,2at2iXat7eL4CM0zarqK8N,2eJjz0p8bitf4XaLWS6i9x,6HLqcNZsDMMipg1W2Kpm7B,26igTnZ9l79lpXxXrjjJ6J,3VdRZjiFa7Ldk6YktenpzY,2B7H8fSkEn99gqohQorqyy,5nzp38RlGTJEr5T92ZggzO,1WGNLf23VZ0G36voHVtmuF,5ccWHhBj7BsHbjcD87sI2Z,7aNN64GAIH6mOdrI4BvleW,35DOOSNdXPYHeUuAFRXGPP,4ViSnqH8yyvbLkXFayntzb,5Y1E98lB23n4lHFEhTBQUN,0exS3bU0KzacoyBcWJx4h4,5KkI0hWsK77sZutSCMsYNd,41MRUxLJyCz2pRWO6wH5cI,0hpoPHsATfiiLdS347dwE9,74tppgg296k2aYhqDTOklg,2qoGnuvGhUhp0Clma6g4MC,4OvAch2E6am0YTa3DrbOZ6,7pvp914OZpWKG4kQkJn4fv,0ug5P22nGDQgcO1VDwuc06,3a7e2hJ3v8a5m8U7pNSqOz,3dHocRlQCjnUXiXFGRMI4A,4uYzE6hhfO96omrWIsQ55w,00kFjhJAcWiTwR7G31p7yO,50TnXxdAXEMWmnfcOJJsi1,40n5o5TvOkZG32p40brIbQ,3ZjySEo9CM0jQ9FoEx4e5o,2GapPFrRJ9qoPbtl9UmwMt,0enY9WM7mmz2WcihSmqmNJ,5JRPcG1umq7ben4n3rTaa1,2M6N1Aa1ufTEYXeZI4XhwB,394QsegImLg6rfrkXtc9Gh,1zzJo23H4Ua4pODaX3haQ8,7LHmMhQV52IoXafWgyFs7q,0nYEUG1vQfxUUXfA0PtDwz,5ODKpgO9XHHy0vm7hgkfQ9,4b9BciN7N6uXexKjivRJZX,1Lw9nfHv7M9sFMEw2P4lrw,6UVga4SyddyJtxR5RW0zDP,6ejpSLKP3DdMoJWbGmxEoH,5dcQmrhhySV4KT01qyPB5s,5egVevVBN9tVTfBGb75gDU,233bg8Vs572stoe3si1iwC,0n0i0HKac7RGWtr8rzuqyV,5PDQVJlxgM7Om2OsmjTXv2,4alo0DClYHZqsKbOHYWPZW,5Kzje01A4PeS1xDqbCjtMK,5qj3LuRlxqEKNjzI6oikEh,0eVg9Z4IRDBcRHPFSg4woC,3BJuNHm0vHKFWt9Hhv9aqu,7u4o09gjTh0OzQvBj4nQnS,7ip0sY03hgqQ0jOI3YcWbv,5Z7kqO1zM3PvcNLKwiovAA,3CxMfTkQZXHx9mr83v87mv,0zs5nMk8kl4cnz6aUEQYQJ,56Uqeouqi3GfiMkNtK3IhX,77BDK5AqhEH1hLo2zZDSc9,2SMTkmKOpnh36rJJCQIQxy,2KYfq3rKLnRNunW1dndmjo,2SkZZ2oRvXCXqxyas2BKKA,65Lx02FHmJyxjJhHm7AbHV,7L0GQ4pwhHzZCtlt6fWZGy,523ys1gE7XLY1lSb4B4WqX,2Gygmpi5S2QI1GJCTQGdeG,54Z1D2VfRlI82XAjIrdMo2,3VcGGjkXJgZ3vu3VkAYlNQ,4Pq3wFnZnri9GvKcVRw44k,7MHK8zVyZTYVNC45p1ddWO,3m4CUolUSHyicSXN0MmQe4,2RkYmczDzC1nNkL5tCpyun,4ecMbdZCevBzWp3dR2vhs5,13nfmaKmRCJXZ4qIQ4RrIu,0Wd0EDdijVskBAGeoz3pNe,0PezBmOmEk7pYJTtNRk4fw,0vxp52Y8kZgAreBstETgOm,4Ndm0rWegzBES4Fc3fxLmP,4k3i6YYSblrKQIcuO5PLmO,4aQcwhliDd1Xy9fdJPFfc6,1Vmi7DJghW9rFojgoo6OqA,0IRodR8uYd7QuM7Z8a9zdk,7rN12lEaRBMVsSUU2EDpml,7hAtPgFCXIICXz6swB8sHB,5VjgxgxkbPJLmI3QyNcaI3,0y6nE8iH3FQl6IMNhAVAeD,3unbgLnXmTSRoPg4TaVJ7n,3GqufTXQspgcBy1N1rZM9f,3S6SH7sp50UTsc2tZWPzPP,157NKEaEFPVRKZvXnYNRiu,61zVyrhTtxa0tWUNANo3VR,3Y6Oz3x7UeiDGNqT2hpekg,0mRLwk1PCR9RUTOoRQOdza,7t6MHFZy37YcufPEHXhpZ6,75ip4qWZY4u2zNi5QO9MDf,3bK8gyqsSsnqm3pWoI4Bku,1DmT1ZYNxm77Uehr1Jofc9,46L3EWj2Zqhd8sbq9gz3sd,4iZspBfW5hdSz0ihjDl9N2,2p5U8fXFfnE0SZZzrDN8CY,17HNE8RXpLNTjuhGrr7uHa,1ry1tq9NuMZcyDKg1Tjg4o,6c1IkcoBpvmnLpaQ921CxV,2mqOeVxD4W7g69yd4vNiFK,3qdP8SO94TUuYprbd5QcKK,3OcR2V45pwiJ6zIphFj4xQ,43Kezp885AOQEGxy9OXXQA,1SjShRIR50an5Q9PdZpS3u,6tNv6bLmOEfVSONTQlrzfA,5sAN8JYpCtsVScxh9rlsgG,0bMn40CrjWqxmWufqeE248,0B5KAP9Kus2ZYmaw4S2Rnt,6bM4flR3k0dNYKudQYZMd3,0dFSYF92KTIf0REPmhxUwp,3FtZ3xuRwzIeFilyQHzozl,6hHCpKDUWnmRxkDW6b3C8k,2pl30PtRBp9lOOVKMO1xTU,2Ld8R6LsANtx5jCaFlI3fY,6qfLv2X2U3MWE68TET2ZU5,2M5OWigT5dgBN2bZSPu67M,1NXq0Oeq3X1NjVYwmXlYnE,2qBRTcB4Cdy4tPh6T4ZeZV,4GYfgipOcm3bjCFcs5ux5u,6OMdiCPDkq0ZhGTUG6h5kM,1RBlJECUnOm56kCt0xI0Ro,3qMjCMVoH12F7XR7m1ic1V,210whiyXeQtMrlw4bD87Da,6gYh8MkAh0iLQO2Uk3zigK,176ZGVZvtRJnZNbm7FfLR0,6GrVoPt8qdlam9VaPtpH5H,2jwJCUYsNS5QfuNjBTJ6G6,1uE2IGV7Tr0lrhVoCYpSWk,6BcjcX59w5uecBvduLjqqD,559fKT5Finw8ze4peJk3XV,3H2aHCkRl3SLWAgjAOnGev,32TPXI5bLWWlj8O5Rujrfk,3alCNs3x9OQh7LzEDMjLCs,5xmXGKBgy7RbBfuIIpJdly,1zcPp2GmXZCproyWSqoAci,0xqwnPrwnA863X4KxmFnBN,0FavtMGr3CAQ70MSxlbbnO,1hpOnh4szoVmmkxcR0k5a5,65AzrfB5RBJBVg2eRCFyjZ,2rRBDkwfaW2zSTtxU5Cxva,0TfsbfAHbBx5OAMcfXlSVk,1N2EeTiwP581PL3Grd7gGI,1RoXiuKXxiBFyyuFkRkEyR,5OK04EayvpNlsQ4OLEGbNG,26vhY3w4KQ747cU8TCvlzc,0SJt9KyheF6VQOIumKYVVw,1UOg7U0B9pUQJ0ksirGzdz,0t55QlYs5dMMhhVPRTElPN,1BmSu3yBFRHuCNHUFqvlg0,5SMrX7TqM1g9aPUSAa9Oos,65SaqPo1pDErk3iWAeH17B,3bVMAKvZ8tBKuk2GaKMivy,2ZJw3o5NL5MKbbEoSvqAgA,3hTcYfgg5eS9WtgZ6iTknT,0dLaqSn32UC4uoruIcwrd0,3S5LAeiiv6phIDY90HiALR,56QlVEslWIXwR0LgseDhXF,6lyNlkCo5gujRIazaGHnOV,1kBw58hHDMZYUcCOaltjrP,2a139qK1MCVQhaGeL2bg1x,4zqTLHcVLzmGB8wY1xVcgX,76wliReEYvjjZKYY0yoGIB,0DUgf6hhD5GxjRuPOLLvKP,0mDU1owcIg30bleqobpXQe,1uZornko05JRVDV16MEM5r,44cjQdlrpVfdI8NijXS8Yf,1QPrZHrdfejaCa2GfkLlBC,4yQrDKxhN1RXBCHhC8yOQ0,3q4LlvKs2yigBnDrU3Mko5,3jbMOfbBZeGbNXWo7OyEL0,4QsCDJ6cL01cPnmLiJPSi5'.split(',')



In [30]:
len(lst)

204